In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob

In [2]:
fnames = glob('./csv/*.csv')
print(fnames)
print(len(fnames))

['./csv\\0.5202_bigru_max_focal.csv', './csv\\0.7162_bigru_latt_max_focal.csv', './csv\\0.7286_deep_bigru_batt_max_focal.csv', './csv\\0.7354_fat_bigru_batt_max_focal.csv', './csv\\0.7418_bigru_lnatt_max_focal.csv', './csv\\0.7426_bilstm_batt_mean_focal.csv', './csv\\0.7458_bigru_batt_drop_max_focal.csv', './csv\\0.7458_bigru_bnatt_max_focal.csv', './csv\\0.7466_bigru_batt_5_max_focal.csv', './csv\\0.7474_bigru_batt_mean_focal.csv', './csv\\0.7490_bilstm_batt_5_max_focal.csv', './csv\\0.7512_bigru_batt_max_bce.csv', './csv\\0.7516_thin_bigru_batt_max_focal.csv', './csv\\0.7522_bilstm_batt_max_focal.csv', './csv\\0.7524_bigru_batt_max_focal.csv', './csv\\0.7554_bilstm_batt_max_bce.csv']
16


In [3]:
THRESHOLD = 0.75
del_fnames = []
for fname in fnames:
    recall = float(fname.replace('./csv\\', '').split('_')[0])
    if (recall < THRESHOLD):
        del_fnames.append(fname)
fnames = [x for x in fnames if x not in del_fnames]
print(fnames)
print(len(fnames))

['./csv\\0.7512_bigru_batt_max_bce.csv', './csv\\0.7516_thin_bigru_batt_max_focal.csv', './csv\\0.7522_bilstm_batt_max_focal.csv', './csv\\0.7524_bigru_batt_max_focal.csv', './csv\\0.7554_bilstm_batt_max_bce.csv']
5


In [4]:
NUM_OPTIONS = 100
TOP_AT = 10

In [5]:
# Cache all predictions to numpy array.
num_files = len(fnames)
num_rows = pd.read_csv(fnames[0])['Id'].values.shape[0]
preds = np.zeros((num_files, num_rows, NUM_OPTIONS)).astype(np.uint8)
for f, fname in enumerate(fnames):
    Predict = pd.read_csv(fname)['Predict'].values.astype(str)
    for r, str_labels in enumerate(Predict):
        labels = str_labels.split('-')[:NUM_OPTIONS]
        for c, label in enumerate(labels):
            if np.isnan(float(label)):
                continue
            preds[f, r, c] = int(label)

In [6]:
sum_preds = np.sum(preds, axis=0)
print (sum_preds.shape)

(1000, 100)


In [7]:
best_ids = np.argsort(sum_preds, axis=-1)

In [8]:
# Create submit
submit = pd.read_csv(fnames[0])

ensembled_labels = []
for r, name in enumerate(submit['Id']):
    ensembled_label = np.zeros(NUM_OPTIONS).astype(np.uint8)
    ensembled_label[best_ids[r, -TOP_AT:]] = 1
    ensembled_labels.append(''.join(['1-' if x else '0-' for x in ensembled_label]))
submit['Predict'] = ensembled_labels
submit.to_csv('Ensemble_{}.csv'.format(THRESHOLD), index=False)